## 3.15 Homework

### Dataset

In this homework, we will continue the New York City Airbnb Open Data. You can take it from
[Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

We'll keep working with the `'price'` variable, and we'll transform it to a classification task.

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error




In [2]:
cols = ['neighbourhood_group','latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

In [3]:
df = pd.read_csv('/Users/luis/Desktop/Py/mlbookcamp-code/course-zoomcamp/03-classification/AB_NYC_2019.csv', usecols=cols)

### Features

For the rest of the homework, you'll need to use the features from the previous homework with additional two `'neighbourhood_group'` and `'room_type'`. So the whole feature set will be set as follows:

* `'neighbourhood_group'`,
* `'room_type'`,
* `'latitude'`,
* `'longitude'`,
* `'price'`,
* `'minimum_nights'`,
* `'number_of_reviews'`,
* `'reviews_per_month'`,
* `'calculated_host_listings_count'`,
* `'availability_365'`

Select only them and fill in the missing values with 0.

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
strings = list(df.dtypes[df.dtypes == 'object'].index)
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [5]:
df.head().T

,0,1,2,3,4
neighbourhood_group,brooklyn,manhattan,manhattan,brooklyn,manhattan
latitude,40.64749,40.75362,40.80902,40.68514,40.79851
longitude,-73.97237,-73.98377,-73.9419,-73.95976,-73.94399
room_type,private_room,entire_home/apt,private_room,entire_home/apt,entire_home/apt
price,149,225,150,89,80
minimum_nights,1,1,3,1,10
number_of_reviews,9,45,0,270,9
reviews_per_month,0.21,0.38,NaN,4.64,0.1
calculated_host_listings_count,6,2,1,1,1
availability_365,365,355,365,194,0


In [6]:
df.reviews_per_month.fillna(0, inplace=True)

In [7]:
df.isna().sum()

neighbourhood_group               0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column `'neighbourhood_group'`?

Sol: Manhattan

In [8]:
#calculate the mode for column 'neighbourhood_group'
df['neighbourhood_group'].mode()

0    manhattan
dtype: object

### Split the data

* Split your data in train/val/test sets, with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to 42.
* Make sure that the target value ('price') is not in your dataframe.

In [9]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
   * In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Example of a correlation matrix for the car price dataset:

<img src="images/correlation-matrix.png" />

Sol: 1.- number_of_reviews, reviews_per_month, 2.- calculated_host_listings_count, availability_365

In [10]:
#create a correlation matrix
corr = df_train.corr()
corr.style.background_gradient(cmap='coolwarm')

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


### Make price binary

* We need to turn the price variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the price is above (or equal to) `152`.

In [11]:
df_above = df.copy()
df_above['above_average'] = np.where(df_above.price >= 152,1,0)

In [12]:
df_above = df_above.drop(['price'], axis=1)

In [13]:
df_train_full, df_test = train_test_split(df_above, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values



### Question 3

* Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
* Which of these two variables has bigger score?
* Round it to 2 decimal digits using `round(score, 2)`

Sol: room_type

In [15]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_train_full.above_average)

In [16]:
categorical = df_train_full.dtypes[df_train_full.dtypes == 'object'].index

In [25]:
cat = list(categorical)

In [19]:
mi = df_train_full[categorical].apply(mutual_info_churn_score)
round(mi.sort_values(ascending=False),2)

room_type              0.14
neighbourhood_group    0.05
dtype: float64

### Question 4

* Now let's train a logistic regression
* Remember that we have two categorical variables in the data. Include them using one-hot encoding.
* Fit the model on the training dataset.
   * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
   * `model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [57]:
num = df_train_full.dtypes[df_train_full.dtypes != 'object'].index
num = list(num)

In [58]:
train_dict = df_train[cat + num].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)


In [59]:
X_train #numpy.ndarray

array([[  0.,  50.,  13., ...,   1.,   0.,   0.],
       [  0.,   7.,   1., ...,   0.,   1.,   0.],
       [  0.,   0.,   1., ...,   1.,   0.,   0.],
       ...,
       [  1.,  88.,   1., ...,   0.,   1.,   0.],
       [  0.,   0.,   1., ...,   0.,   1.,   0.],
       [  0., 281.,   2., ...,   1.,   0.,   0.]])

### OHE

In [43]:
X_train_num = df_train[num].values

scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train_num)

In [45]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

X_train_cat = ohe.fit_transform(df_train[cat].values)

In [46]:
ohe.get_feature_names()

/Users/luis/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_bronx', 'x0_brooklyn', 'x0_manhattan', 'x0_queens',
       'x0_staten_island', 'x1_entire_home/apt', 'x1_private_room',
       'x1_shared_room'], dtype=object)

In [60]:
X_train = np.column_stack([X_train_num, X_train_cat])

In [61]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

### Check accuracy in df_val (2 methods)

In [64]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

val_dict = df_val[cat + num].to_dict(orient='records')
X_val = dv.transform(val_dict)

y_pred = model.predict(X_val)

accuracy = np.round(accuracy_score(y_val, y_pred),2)
print(accuracy)

0.57


In [62]:
X_val_num = df_val[num].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[cat].values)

X_val = np.column_stack([X_val_num, X_val_cat])

In [63]:
y_pred = model.predict_proba(X_val)[:, 1]
accuracy_score(y_val, y_pred >= 0.5)

1.0

### Question 5

* We have 9 features: 7 numerical features and 2 categorical.
* Let's find the least useful one using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `neighbourhood_group`
   * `room_type` 
   * `number_of_reviews`
   * `reviews_per_month`

> **note**: the difference doesn't have to be positive

In [53]:
features = cat + num
features

['neighbourhood_group',
 'room_type',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'above_average']

In [ ]:
orig_score = accuracy #save the original accuracy score


for c in features:  #for each feature
    subset = features.copy()    #create a copy of the features
    subset.remove(c)        #remove the feature from the list
    
    train_dict = df_train[subset].to_dict(orient='records') #create a list of dictionaries

    dv = DictVectorizer(sparse=False)   #create a DictVectorizer
    dv.fit(train_dict)  #fit the DictVectorizer to the list of dictionaries

    X_train = dv.transform(train_dict)  #transform the list of dictionaries into a numpy array

    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)  #create a LogisticRegression model
    model.fit(X_train, y_train) #fit the model to the training data

    val_dict = df_val[subset].to_dict(orient='records') #create a list of dictionaries
    X_val = dv.transform(val_dict)  #transform the list of dictionaries into a numpy array

    y_pred = model.predict(X_val)   #predict the labels of the validation data

    score = accuracy_score(y_val, y_pred)   #calculate the accuracy score
    print(c, orig_score - score, score) #print the feature, the difference in accuracy, and the accuracy score

### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'price'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

In [66]:
train_dict = df_train[cat + num].to_dict(orient='records')  #create a list of dictionaries

dv = DictVectorizer(sparse=False)   #create a DictVectorizer
dv.fit(train_dict)  #fit the DictVectorizer to the list of dictionaries

X_train = dv.transform(train_dict) #transform the list of dictionaries into a numpy array

val_dict = df_val[cat + num].to_dict(orient='records')  #create a list of dictionaries
X_val = dv.transform(val_dict)  #transform the list of dictionaries into a numpy array

In [68]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a,random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    
    print(a, round(score, 3))

0 0.0
0.01 0.0
0.1 0.0
1 0.0
10 0.001
